In [28]:
import numpy as np, pandas as pd
from datetime import datetime

At this moment we have 3 .csv files for EACH platform:

- Games information
- Critics reviews
- Users reviews

The objective is to have just 3 cleaned .csv files in total: 
1. Games information for all platforms
2. Meta Reviews for all platforms
3. User Reviews for all platforms

In [29]:
# Small helper function

def obj_to_numeric(df, cols):
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df.loc[df[col].isnull(), col] = 0

## Games info

This code loads the csv for each platform that we obtained.
- fills up meta score and user scores with "no score yet" in case there is none.
- fixes the dates and drop rows with no release date
- removes the _summary_ column which was not used (the extract script should be fixed to take that into account) 

In [30]:
platforms = ['wii-u','ps4','xbox-series-x','ps5','switch','3DS','xboxone']



for platform in platforms:
    
    # reading the CSV
    df = pd.read_csv('raw_data/'+ platform + '_games.csv', lineterminator='\n')
    
    # Removing the summary column - it is already non-existent in the new version of the scraper
    #df=df.drop(['summary'],axis=1)
    
    # Converting scores anc counts
    obj_to_numeric(df, ['user_score', 'user_pos', 'user_mixed', 'user_neg'])
    df.loc[df['meta_overview'].isnull(), 'meta_overview'] = 'No score yet'
    df.loc[df['user_overview'].isnull(), 'user_overview'] = 'No user score yet'
    df['n_user_score'] = df['user_score'] * 10
    
    # Converting dates to actual time objects
    df['real_date'] = pd.to_datetime(df['release_date'],format="%b %d, %Y", errors='coerce')
    df = df.dropna(subset=['real_date'])
    
    # Saving the file
    df.to_csv('raw_data/'+ platform + '_games_cleaned.csv', index=False, encoding = 'utf-8')

    print(platform)
    #print(df.describe().loc[['min', 'max','count'], ['meta_score', 'user_score']])

wii-u
ps4
xbox-series-x
ps5
switch
3DS
xboxone


And we save the resulting dataframe into **games.csv**

In [31]:
tables = [pd.read_csv(f'raw_data/{c}_games_cleaned.csv', lineterminator='\n') for c in platforms]
df = pd.concat(tables)
df.to_csv('data/games.csv', index=False, encoding = 'utf-8')

## Reviews

#### Meta reviews

Load critics reviews of each platform

In [32]:
meta_reviews = [pd.read_csv(f'raw_data/{c}_meta_reviews.csv', lineterminator='\n') for c in platforms]

In [33]:
for t in meta_reviews: 
    print(t.shape)

(5930, 5)
(60727, 5)
(5869, 5)
(11230, 5)
(39972, 5)
(12872, 5)
(27095, 5)


Removing rows with missing values

In [34]:
df = pd.concat(meta_reviews)
df = df.dropna()

Fixing date

In [35]:
df['real_date'] = pd.to_datetime(df['date'],format="%b %d, %Y", errors='coerce')
df = df.dropna(subset=['real_date'])


Saving

In [36]:
df.to_csv('data/meta_reviews.csv', index=False, encoding = 'utf-8')

#### User reviews

In [37]:
user_reviews = [pd.read_csv(f'raw_data/{c}_user_reviews.csv', lineterminator='\n') for c in platforms]
# for t in user_reviews: print(t.shape)

df = pd.concat(user_reviews)
df = df.dropna()
df['real_date'] = pd.to_datetime(df['date'],format="%b %d, %Y", errors='coerce')
df = df.dropna(subset=['real_date'])

# Fixing the scores to put them on the same scale as the Meta reviews
df['score']=df['score']*10

# Saving into a CSV
df.to_csv('data/user_reviews.csv', index=False, encoding = 'utf-8')